<a href="https://colab.research.google.com/github/1900690/gradio/blob/main/gradio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Google Colab上でwebアプリが作れるGradioの使い方](https://clameyes.com/posts/google-colab-web-app-gradio)

[garadioのplaygroundにサンプルがたくさん](https://www.gradio.app/playground)

##※認証がある場合はID;test pass:gradio

In [ ]:
#@title gradioライブラリをインストール
#@markdown この共有リンクの有効期限は1週間です。永続的なホスティングとGPUのアップグレードを無料で行うには、作業ディレクトリのターミナルから `gradio deploy` を実行し、Hugging Face Spaces (https://huggingface.co/spaces) にデプロイしてください。
!pip -q install gradio

In [ ]:
#@title 指定したテキストを返すアプリ
import gradio as gr

# 関数を定義
def greet(name):
    return "Hello " + name + "!"

# Webアプリを作成
app = gr.Interface(fn=greet, inputs="text", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2db3a0419addaceb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title スライダーで数を定義するアプリ
import gradio as gr

# 関数を定義
def count(number):
    return "数字は " + str(number) + "!"

# Webアプリを作成
app = gr.Interface(fn=count, inputs="slider", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d38daa5056d6f920f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Google ColabでGUI｜gradioを利用したウェブアプリの作り方を解説](https://tech.aru-zakki.com/gradio-on-google-colab/)

In [ ]:
#@title 画像をフィルターに通すアプリ
import numpy as np
import gradio as gr

#セピアフィルターを定義
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

app = gr.Interface(sepia, gr.Image(), "image")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://402e2f7432d4e668b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title　入力した値を覚えるアプリ
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

app = gr.Interface(
    track_score,
    gr.Number(label="Score"),
    gr.JSON(label="Top Scores")
)
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd3b70f4d607c1eddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#アナログメータ針検出アプリをgradio化してみると

In [ ]:
import cv2
import math
import numpy as np
import gradio as gr

def analog(input_img):
        # 指定する中心点（メータの中心）
        cx, cy =(346,304)#横は→に進む、縦は↓に進む
        angle_off_set=-133#pf0の時に０になるように初期値を設定する

        # グレースケール変換 & 二値化
        gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        threshold = 100
        _, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

        # エッジ検出
        edges = cv2.Canny(img_thresh, 50, 200, apertureSize=3)

        # Hough変換で直線を検出（2本の直線を得る）
        minn = None
        for m in range(10, 161, 1):
            lines = cv2.HoughLines(edges, 1, np.pi / 180, m)
            if lines is not None and len(lines) == 2:
                minn = m
                break

        # 直線が2本見つからなかった場合のエラーハンドリング
        if minn is None:
            raise ValueError("直線が2本検出されませんでした。画像のコントラストやエッジ検出のパラメータを調整してください。")

        lines = cv2.HoughLines(edges, 1, np.pi / 180, minn)

        theta_t = []  # 直線の角度
        coefficients = []  # 直線の傾きと切片

        for i in range(len(lines)):
            for rho, theta in lines[i]:
                theta_t.append(theta)

                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho

                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(input_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # x2-x1が0の場合のエラーハンドリング
                if x2 - x1 == 0:
                    continue  # 無限傾斜を避ける
                a0 = (y2 - y1) / (x2 - x1)
                b0 = y1 - a0 * x1
                coefficients.append((a0, b0))

        # 直線の数をチェック（2本以上ある場合は最初の2本を使用）
        if len(coefficients) < 2:
            raise gr.Error("十分な直線が検出されませんでした。")

        (a1, b1), (a2, b2) = coefficients[:2]  # 最初の2本を選択

        # 交点を計算
        if a1 == a2:
            raise gr.Error("平行な直線のため交点が計算できません。")

        x_t = (b2 - b1) / (a1 - a2)
        y_t = a1 * x_t + b1

        # 交点と中心点を結ぶ線の角度を計算
        angle_rad = np.arctan2(y_t - cy, x_t - cx)
        angle_deg = np.degrees(angle_rad)
        angle =int(angle_deg) +angle_off_set
        angle=angle*-1  #負の角度を修正
        Kpa=8/27*angle
        if Kpa * 10.197 > 0:
          PH=math.log10(Kpa*10.197)
        elif Kpa == 0 :
          PH=0
        else:
          PH=-1
          raise gr.Error("角度がマイナスになりました")

        # 結果を画像に描画
        img_temp=cv2.circle(input_img, (int(cx), int(cy)), 10, (0, 255, 0), -1)  # 中心点を描画
        img_temp=cv2.circle(img_temp, (int(x_t), int(y_t)), 10, (0, 255, 0), -1)  # 交点を描画
        img_temp=cv2.line(img_temp, (cx, cy), (int(x_t), int(y_t)), (255, 0, 0), 2)  # 中心と交点を結ぶ線
        img_temp=cv2.putText(img_temp, f"{PH:.2f} ph", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        return img_temp

app = gr.Interface(
    analog,
    gr.Image(),
    "image",
     examples=[
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/000.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/021.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/026.jpg"]
     ],
    title="アナログメータ針検出アプリ",
    description="アナログメーターの画像をアップロードしてください。画像からから針の角度を検出します。",
    allow_flagging='never'
    )
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1d3ce4eeec00673aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#開空率測定アプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr


def detect_sky(input_img, Rmax, Rmin, Gmax, Gmin, Bmax, Bmin):
    # Gradio からの画像は RGB 形式
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    # 各色を取り出し
    r = np.array(src[:, :, 2], dtype=np.int16)
    g = np.array(src[:, :, 1], dtype=np.int16)
    b = np.array(src[:, :, 0], dtype=np.int16)

    # 色フィルタリング
    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binde_image = np.zeros_like(src)
    binde_image[:, :, 0] = b_mask
    binde_image[:, :, 1] = g_mask
    binde_image[:, :, 2] = r_mask

    # バイナリマスク作成
    binde_image_binary = (r_mask + g_mask + b_mask)
    binde_image_binary = np.where(binde_image_binary > 0, 255, 0).astype("uint8")

    # 輪郭抽出
    contours, _ = cv2.findContours(binde_image_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = np.copy(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_line = cv2.drawContours(img_with_line, contours, i, (0, 0, 255), 1)

    # 可視化用にRGBに変換
    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # 面積マスク作成
    img_with_area = np.zeros_like(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_area = cv2.fillPoly(img_with_area, [contours[i][:, 0, :]], (255, 255, 255))

    img_with_area_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype("uint8")
    img_with_area_rgb = np.stack([img_with_area_binary]*3, axis=2)

    # 占有率計算
    all_pixel = img_with_area_binary.size
    sky_pixel = np.sum(img_with_area_binary > 0)
    sky_occupancy_rate = int((sky_pixel / all_pixel) * 100)

    return f"空の面積率: {sky_occupancy_rate}%",img_with_line_rgb, img_with_area_rgb

# Gradioインターフェース
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="画像"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="赤上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="赤下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="緑上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="緑下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="青上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="青下限"),
    ],
    outputs=[
        gr.Label(label="空の面積率"),
        gr.Image(label="空の輪郭表示"),
        gr.Image(label="空の領域マスク")
    ],
    examples=[
    ["https://github.com/1900690/grape-sky-segmentation/releases/download/sample/sample.jpg", 255, 200, 255, 200, 255, 200]
    ],
    allow_flagging='never',
    title="画像から開空率測定アプリ",
    description="画像の空にあたる色をRGBで指定することで空の割合を計算します。",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)


app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069578a63bc76b58c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#ナスLAIアプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# ====== リサイズユーティリティ ======
    """縦横比を保って画像を指定サイズ以下にリサイズする関数を定義"""
def resize_with_aspect_ratio(image, target_width=1108, target_height=1478):
    #入力画像の高さhと幅wを取得する
    h, w = image.shape[:2]
    #幅方向のスケール倍率（目標幅に合わせるための倍率）を計算。
    scale_w = target_width / w
    #高さ方向のスケール倍率（目標高さに合わせるための倍率）を計算。
    scale_h = target_height / h
    #幅・高さ両方を満たすため小さい方の倍率を選ぶ（縦横比維持）。
    scale = min(scale_w, scale_h)
    #新しい幅を整数化して計算。
    new_w = int(w * scale)
    #新しい高さを整数化して計算。
    new_h = int(h * scale)
    #OpenCVで実際にリサイズ（INTER_AREAは縮小時に適した補間）。
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    #リサイズ済み画像を返す。
    return resized

# ====== 画像解析ユーティリティ ======
    """HSV画像から自動で葉色のHue範囲を推定して返す関数を定義"""
def auto_leaf_range_from_hist(hsv, s_thresh=40, v_thresh=40):
    #彩度と明度が閾値を超えるピクセルだけを真にするブールマスクを作成
    mask_sv = (hsv[:, :, 1] > s_thresh) & (hsv[:, :, 2] > v_thresh)
    #条件を満たすピクセルのHue（色相）値のみ抽出
    hue_values = hsv[:, :, 0][mask_sv]
    #Hue値のヒストグラムを180の区画範囲で作成（hueは0～180まで）。
    hist_h, _ = np.histogram(hue_values, bins=180, range=(0, 180))
    #ヒストグラムのピークを検出。ピークの高さは最大値の30%以上を条件にしている。
    peaks, _ = find_peaks(hist_h, height=np.max(hist_h) * 0.3)
    #ピークが検出できなかった場合の条件分岐,ピーク無しならデフォルトの下限/上限（Hue:25〜90, S:40〜255, V:40〜255）を返す。
    if len(peaks) == 0:
        return np.array([25, 40, 40]), np.array([90, 255, 255])
    #検出したピークのうち最も小さいインデックス（最も左のピーク）を選ぶ
    main_peak = peaks[np.argmin(peaks)]
    #主ピークの左側に15の余裕を取りつつ下限を0未満にならないように制限。
    hue_min = max(0, main_peak - 15)
    #主ピークの右側に15の余裕を取りつつ上限を179を超えないように制限。
    hue_max = min(179, main_peak + 15)
    # 推定した下限・上限のHSV配列（Hue範囲とS/Vの固定閾値）を返す。
    return np.array([hue_min, 40, 40]), np.array([hue_max, 255, 255])

    """Hue値配列からヒストグラム画像を作る関数を定義"""
def generate_hue_histogram(hue_values, h_min, h_max):
    #与えられたHue値で180区画範囲のヒストグラムを作成。
    hist_h, _ = np.histogram(hue_values, bins=180, range=(0, 180))
    #matplotlibで図と軸を作成（幅6インチ×高さ3インチ）。
    fig, ax = plt.subplots(figsize=(6, 3))
    #ヒストグラムを線グラフとしてプロット（色は緑）。
    ax.plot(hist_h, color='g')
    #ヒストグラム上の選択範囲（h_min〜h_max）を半透明の黄色で塗る。
    ax.axvspan(h_min, h_max, color='y', alpha=0.3, label='selection range')
    #グラフタイトルを設定。
    ax.set_title('Hue Histogram')
    #凡例を表示
    ax.legend()
    #図のレンダリングを実行（キャンバスに描画）。
    fig.canvas.draw()
    #レンダラのRGBAバッファをNumPy配列に取り出し、画像として扱える形にする。
    hist_img = np.array(fig.canvas.renderer.buffer_rgba())
    # メモリ節約のために図を閉じる。
    plt.close(fig)
    #RGBA画像配列を返す
    return hist_img

    """入力画像とHSV閾値を受け取り、マスク生成・LAI計算・ヒストグラム出力を行う主要処理関数を定義。"""
def process_image(image, h_min, h_max, s_min, v_min):
    #まず画像を関数でリサイズして扱いやすくする。
    image = resize_with_aspect_ratio(image)
    #Gradioからの画像はRGBなので一度BGRに変換し、その後HSV色空間に変換してhsvを作る（OpenCVはBGR基準）。
    hsv = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2HSV)

    #マスク下限のHSV配列を作成。
    lower_bound = np.array([h_min, s_min, v_min])
    #マスク上限（Hueはh_max、S/Vは255固定の上限）を作成。
    upper_bound = np.array([h_max, 255, 255])
    #指定HSV範囲に入るピクセルを白(255)、それ以外を黒(0)にしたバイナリマスクを作成。
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    #参照画像（キャリブレーション）でのピクセル数を格納する定数:基準画像のピクセル数（picasosから抽出）
    reference_pixel_count = 1636516
    #参照時の1ピクセル当たりの面積（単位不記載）を格納する定数:基準画像の１ピクセル当たりの面積（picasosから抽出）
    area_per_pixel_ref = 2.170139

    #マスクで白（対象）になっているピクセル数を数える（葉のピクセル数）。
    leaf_pixels = int(np.count_nonzero(mask))
    #マスク全体のピクセル総数（画像サイズに相当）を取得。
    total_pixels = mask.size
    # LAIの値を計算する式：葉ピクセル数×ピクセル面積（百万でm2）に、画像サイズ比（total_pixels / reference_pixel_count）を掛けて補正している。
    LAI_rate = (leaf_pixels * area_per_pixel_ref) / 1_000_000 * (total_pixels / reference_pixel_count)
    #計算結果を短い文字列に整形（有効数字3桁相当の表示）。
    result_text = f"葉のLAI: {LAI_rate:.3g}"

    #彩度・明度が40より大きいピクセルのHue値を抽出（ヒストグラム作成用）。
    hue_values = hsv[:, :, 0][(hsv[:, :, 1] > 40) & (hsv[:, :, 2] > 40)]
    #上で抽出したHue値と現在のHue閾値でヒストグラム画像を生成。
    hist_img = generate_hue_histogram(hue_values, h_min, h_max)

    #計算したLAIの文字列,スクをGray→RGB変換して3チャンネル画像としたもの,作成したヒストグラム画像（RGBA配列）を返す。
    return (
        result_text,
        cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB),
        hist_img
    )

    """画像を受け取って、スライダーの初期値（自動検出されたHSV閾値）を返す関数を定義。"""
def init_sliders(image):
    #スライダー初期化関数内でもリサイズして一貫性を保つ。
    image = resize_with_aspect_ratio(image)
    #リサイズ済画像をRGB→BGR→HSVへ変換してhsvを作る。
    hsv = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2HSV)
    #先述の自動推定関数で下限と上限のHSV配列を得る。
    lower, upper = auto_leaf_range_from_hist(hsv)
    #スライダー用にHue下限・Hue上限・S下限・V下限を整数で返す（順序はh_min, h_max, s_min, v_min）。
    return int(lower[0]), int(upper[0]), int(lower[1]), int(lower[2])

# ====== Gradio UI ======
"""gradioで表示するためのブロック"""
with gr.Blocks(title="葉のLAI計算アプリ") as demo:
    gr.Markdown("## アップロードした画像からLAIを計算します。")

    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="numpy", label="画像をアップロード")

            btn_run = gr.Button("解析する", variant="primary")

            with gr.Accordion("うまく検出できない場合の微調整用", open=False):
                gr.Markdown("HSV色空間で値を調整する（抽出したHueヒストグラムの山をきれいに挟むように色相の上限と下限を調整してください）")
                h_min_slider = gr.Slider(0, 179, step=1, label="色相の下限")
                h_max_slider = gr.Slider(0, 179, step=1, label="色相の上限")
                s_min_slider = gr.Slider(0, 255, step=1, label="彩度の下限")
                v_min_slider = gr.Slider(0, 255, step=1, label="明度の下限")

            gr.Examples(
                examples=[
                    ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG"],
                    ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg"]
                ],
                inputs=img_input,
                label="画像例",
                cache_examples=False
            )

        with gr.Column():
            json_output = gr.Label(label="LAIの計算結果")
            mask_output = gr.Image(type="numpy", label="葉のマスク画像（葉が白、それ以外黒）")
            hist_output = gr.Image(type="numpy", label="抽出したHueヒストグラムの範囲")

    img_input.change(
        fn=init_sliders,
        inputs=img_input,
        outputs=[h_min_slider, h_max_slider, s_min_slider, v_min_slider]
    )

    btn_run.click(
        fn=process_image,
        inputs=[img_input, h_min_slider, h_max_slider, s_min_slider, v_min_slider],
        outputs=[json_output, mask_output, hist_output]
    )

    gr.Markdown(
        """
        ---
        <small>Copyright (c) 2025 risa iwase
        Released under the MIT license
        <a href="https://opensource.org/licenses/mit-license.php" target="_blank">https://opensource.org/licenses/mit-license.php</a></small>
        """
    )

if __name__ == "__main__":
    demo.launch()

#画像からカラーヒストグラを作成するアプリをgradio化してみると

In [ ]:
import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import os
from uuid import uuid4

def calculate_histograms(image_rgb):
    """RGB画像から各チャンネルのヒストグラムを計算する"""
    colors = ['Red', 'Green', 'Blue']
    hist_data = {
        col: cv2.calcHist([image_rgb], [i], None, [256], [0, 256]).flatten()
        for i, col in enumerate(colors)
    }
    return hist_data

def plot_histogram(hist_data):
    """ヒストグラムデータをプロットしてmatplotlibのFigureを返す"""
    fig = plt.figure(figsize=(6, 4))
    for col in hist_data:
        plt.plot(hist_data[col], color=col.lower())
    plt.title("RGB Color Histogram")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    return fig

def figure_to_image(fig):
    """matplotlibのFigureをPIL画像に変換"""
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    img_np = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape((h, w, 4))
    plt.close(fig)
    return Image.fromarray(cv2.cvtColor(img_np, cv2.COLOR_RGBA2RGB))

def save_histogram_csv(hist_data):
    """ヒストグラムデータをCSVとして保存し、ファイルパスを返す"""
    df = pd.DataFrame(hist_data)
    df.index.name = "Pixel Value"
    csv_path = f"/tmp/histogram_{uuid4().hex}.csv"
    df.to_csv(csv_path)
    return csv_path

def create_histogram_and_csv(image):
    """Gradioから呼び出されるメイン処理"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hist_data = calculate_histograms(image_rgb)
    fig = plot_histogram(hist_data)
    histogram_img = figure_to_image(fig)
    csv_path = save_histogram_csv(hist_data)
    return histogram_img, csv_path

# Gradio インターフェース
demo = gr.Interface(
    fn=create_histogram_and_csv,
    inputs=gr.Image(type="numpy", label="画像をアップロードする"),
    outputs=[
        gr.Image(type="pil", label="RGB ヒストグラム"),
        gr.File(label="ヒストグラムのCSVをダウンロード")
    ],
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG"],
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg"]
    ],
    title="RGBヒストグラム生成ツール",
    description="画像をアップロードしてそのRGBヒストグラムを表示し、またヒストグラムの値をCSVファイルとしてダウンロードできます。",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60524e8d408a298b7f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#画像とそのマスク画像から重なりを抽出するアプリをgradio化してみると

In [ ]:
import gradio as gr
import numpy as np
import cv2

def extract_mask_transparent(image, mask):
    if image is None or mask is None:
        return None

    # 画像が3チャンネル(RGB)であることを保証
    if image.shape[2] == 4:
        image = image[:, :, :3]

    # マスク画像をグレースケール化
    if len(mask.shape) == 3:
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    else:
        mask_gray = mask

    # マスクの白い部分(=255)を不透明、それ以外を透明にしたアルファチャンネルを作成
    _, mask_bin = cv2.threshold(mask_gray, 127, 255, cv2.THRESH_BINARY)
    alpha = mask_bin.astype(np.uint8)

    # RGBA画像を作成
    rgba = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    rgba[:, :, 3] = alpha

    return rgba

iface = gr.Interface(
    fn=extract_mask_transparent,
    inputs=[
        gr.Image(type="numpy", label="元画像をアップロード"),
        gr.Image(type="numpy", label="マスク画像をアップロード")
    ],
    outputs=gr.Image(type="numpy", label="マスク重なり部分"),
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg","https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200mask.jpg"],
    ],
    title="画像とマスクの重なり部分抽出ツール",
    description="画像とマスクの重なり部分を抽出してダウンロードできます",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch()